# PreProcess

### Read DataSet

In [2]:
with open ("hafez.txt", "r") as dataset:
    data = dataset.read().splitlines()

In [ ]:
data

### Read Stop Words

In [3]:
with open ("fa_stop_words.txt", "r") as fa_stop_words:
    stop_words = fa_stop_words.read().splitlines()

In [ ]:
stop_words

### Normalize with Hazm

In [4]:
!pip install hazm

     |████████████████████████████████| 316 kB 6.5 MB/s 
     |████████████████████████████████| 1.4 MB 61.5 MB/s 
     |████████████████████████████████| 233 kB 67.4 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=e5c5eb666d05eb3702840ed250299ed23c078b48dd6354e907ac9070ecbe26b9
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154467 sha256=82e6cb9cac00698d7652aa17d640c57a5fbf57c2fb2e7bc1fc2a47d2bef48ce8
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [5]:
from hazm import *

In [6]:
normalizer = Normalizer()
normalized_data = []
for line in data:
  normalized_data.append(normalizer.normalize(line))

In [ ]:
normalized_data

### Tokenize with Hazm

In [7]:
tokenized_data = []
for line in normalized_data:
  tokenized_data.append(word_tokenize(line))

In [ ]:
tokenized_data

### Remove Stop Words

In [8]:
clear_data = []
for line in tokenized_data:
  clear_line = []
  for word in line:
    if word not in stop_words:
      clear_line.append(word)
  clear_data.append(clear_line)

In [ ]:
clear_data

### Stemming with Hazm
    Finding the root of words

In [9]:
stemmer = Stemmer()
stemmed_data = []
for line in clear_data:
  stemmed_line = []
  for word in line:
    stemmed_word = stemmer.stem(word)
    if len(stemmed_word) > 1:
      stemmed_line.append(stemmed_word)

  stemmed_data.append(stemmed_line)  

In [ ]:
stemmed_data

### convert line arrays to sentence

In [10]:
clear_sentences = []
for line in stemmed_data:
  clear_sentences.append(" ".join(line))

clear_sentences = [item for item in clear_sentences if item]  

In [ ]:
clear_sentences

# Process

### Build the corpus vocabulary

In [ ]:
from keras.preprocessing import text

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(clear_sentences)

word2id = tokenizer.word_index
id2word = {v:k for k, v in word2id.items()}

vocab_size = len(word2id) + 1 
embed_size = 100

wids = [[word2id[w] for w in text.text_to_word_sequence(doc)] for doc in clear_sentences]
print('Vocabulary Size:', vocab_size)
print('Vocabulary Sample:', list(word2id.items())[:10])

### Build a skip-gram generator

In [ ]:
from keras.preprocessing.sequence import skipgrams

skip_grams = [skipgrams(wid, vocabulary_size=vocab_size, window_size=10) for wid in wids]

pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(10):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          id2word[pairs[i][0]], pairs[i][0], 
          id2word[pairs[i][1]], pairs[i][1], 
          labels[i]))


### Build the skip-gram model architecture

In [ ]:
from keras.layers import *
from keras.layers.core import Dense, Reshape
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential

word_model = Sequential()
word_model.add(Embedding(vocab_size, embed_size,
                         embeddings_initializer="glorot_uniform",
                         input_length=1))
word_model.add(Reshape((embed_size, )))

context_model = Sequential()
context_model.add(Embedding(vocab_size, embed_size,
                  embeddings_initializer="glorot_uniform",
                  input_length=1))
context_model.add(Reshape((embed_size,)))

merged_output = add([word_model.output, context_model.output]) 

model = Sequential()
model.add(Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid"))

final_model = Model([word_model.input, context_model.input], model(merged_output))
final_model.compile(loss="mean_squared_error", optimizer="rmsprop")
final_model.summary()

# visualize model structure
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(final_model, show_shapes=True, show_layer_names=False, rankdir='TB').create(prog='dot', format='svg')) 